In [1]:
cd /home/ubuntu/DistMEC/

/home/ubuntu/DistMEC


In [2]:
import numpy as np
import matplotlib.pyplot as plt

from classes.User import *
from classes.solver import *
from classes.Server import *
from classes.distributed_utils import *

import pickle
import os

In [3]:
def load_trial(path_user, path_regret,  trial_num):
    
    user_load_path = path_user + str(trial_num) + '.p'
    with open(user_load_path, 'rb') as handle:
        user_dict = pickle.load(handle)
    
    regret_load_path = path_regret + str(trial_num) + '.p'
    with open(regret_load_path, 'rb') as handle:
        regret_dict = pickle.load(handle)
    
    return user_dict, regret_dict


def track_collision(users):
    
    collision = np.zeros([len(users),len(users)])
    time_steps = len(users[0].history_location)
    for t in range(time_steps):
        arm_pulls = []
        for u in users:
            arm_pulls += [u.history_pull[t]]
        for u in range(len(users)):
            for u2 in range(len(users)):
                if u != u2 and arm_pulls[u] == arm_pulls[u2]:
                    collision[u,u2] += 1
    return collision

def track_movement_overlap(users):
    mvmt = np.zeros([len(users),len(users)])
    time_steps = len(users[0].history_location)
    for t in range(time_steps):
        locs = []
        for u in users:
            locs += [u.history_location[t]]
        for u in range(len(users)):
            for u2 in range(len(users)):
                if u != u2 and locs[u] == locs[u2]:
                    mvmt[u,u2] += 1
    return mvmt

def track_movement(users, num_servers):
    locs = np.zeros([len(users),num_servers])
    time_steps = len(users[0].history_location)
    for t in range(time_steps):
        for u in range(len(users)):
            then_loc = int(users[u].history_location[t])
            locs[u, then_loc] += 1

    return locs


def track_mu_est(users, num_servers):
    mu_est = np.zeros([len(users),num_servers])
    for u in range(len(users)):
        mu_est[u,:] = users[u].mu
    return mu_est

def pull_records(users, num_servers):
    pull_rec = np.zeros([len(users),num_servers])
    for u in range(len(users)):
        pull_rec[u,:] = users[u].pulls
    return pull_rec

In [8]:
# path_regret = 'results/22_06_13_reserve_troubleshoot/regret_trial'
# path_user = 'results/22_06_13_reserve_troubleshoot/user_trial'

path_regret = 'results/22_06_14_reserve_troubleshoot_n3_kick/regret_trial'
path_user = 'results/22_06_14_reserve_troubleshoot_n3_kick/user_trial'


In [9]:
mode = 'new'
idxs = 8
a,b = load_trial(path_user, path_regret, idxs)
coll = track_collision(a[mode])
mvmt_overlap = track_movement_overlap(a[mode])
mu_est = track_mu_est(a[mode], 16)
pull_rec = pull_records(a[mode],16)
mvmt = track_movement(a[mode],16)

# print(coll.astype(int))
# print(mu_est)
# print(mu_est.argmax(axis=1))
print(pull_rec.astype(int))
# print(pull_rec.argmax(axis=1))
# print(mvmt/5000)
# print(mvmt.argmax(axis=1))


[[   1 4984    1    1    1    1    1    1    1    1    1    1    1    2
     1    1]
 [   1    1    1    1    1    1    2    2    1    1 4982    2    1    1
     1    1]
 [   1    1    1    1    1    1    1    2    1    1    1    1    1 4983
     2    1]
 [   1    2    1    2    1    1    1 4983    1    1    1    1    1    1
     1    1]
 [   1    2    2    1    1    1    2    1    4    2    1    1 4973    4
     3    1]
 [   2    3    2    1    2    1    1    2    2    1    2    1    1    2
     2 4975]
 [   1    3    1 4982    2    1    1    1    1    1    1    1    1    1
     1    1]
 [   3    5    3    4 4956    4    2    4    4    1    1    1    6    2
     2    2]
 [   2    3    2    8    2    9    2    3    3    2    2    2    5    3
  4950    2]
 [  39  128  124   22  647   17   18   19  137   13   17   18  512   12
  2767  510]]


In [10]:
print(mvmt)

[[ 497.   42.   61.  401.  968.  130.   17.  114.  465.  102.  470.  348.
   482.  383.   34.  486.]
 [ 303.  426.  289.   85.   58.  123.   86.  189.   91. 1173.  243.  114.
   378.  729.  600.  113.]
 [  17.   29. 1035.   12.   84.  171.  634.   62.  363.  233.  150.  174.
   343.  401.  840.  452.]
 [1156.  484.  427.  321.  586.  207.   33.  164.   33.   32.  214.  128.
   424.  140.  574.   77.]
 [ 271.  468.  227.  173.  267.   79.  111.  100.  331.   26.  340. 1381.
   602.  410.   83.  131.]
 [ 183.   19.  461.  285.   20.  429.  250.  311.  196.  767.    9.  204.
   376.  392.  795.  303.]
 [ 978.  581.  461.   10.  295. 1004.  123.   16.  344.  189.  141.   21.
   127.  162.   79.  469.]
 [  88.  458.  693.  288.  164.   15.   46.  920.   55. 1612.  145.   72.
   133.    6.  268.   37.]
 [ 331.  317.  204.  226. 1065.  105.  551.  326.  326.  105.  204.  697.
    40.  238.   62.  203.]
 [  71.  116.  540.  172.  270.  111.  279.  643.  557.  126.   19.  575.
   106.   60.  88

In [132]:
a[mode][1].history_location

[10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 12,
 12,
 12,
 12,
 12,
 12,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [92]:
a

{'worst': [<classes.User.User at 0x7f240a622c50>,
 'res': [<classes.User.User at 0x7f2409ce2eb8>,
 'new': [<classes.User.User at 0x7f240a7d38d0>,
  <classes.User.User at 0x7f2403393438>]}